In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks'
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
# from tqdm.auto import tqdm
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.autograd import Variable
import torch.optim as optim
import torchvision
# from torchsummary import summary
import gc

dataset_dir = '../dataset'
#dataset_dir = '/kaggle/input/bengaliai-cv19'
train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')

# 前処理を関数にまとめた
def resize(X, out_height=64, out_width=64):
    print('Resizing raw image... / 前処理実行中…')
    resized = {} # 前処理された画像が格納されるリスト

    for i in range(len(X)):
        image = X.iloc[[i]].values.reshape(HEIGHT, WIDTH)
        _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        left = 1000
        right = -1
        top = 1000
        bottom = -1

        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            left = min(x, left)
            right = max(x+w, right)
            top = min(y, top)
            bottom = max(y+h, bottom)

        roi = image[top:bottom, left:right]
        resized_roi = cv2.resize(roi, (out_height, out_width),interpolation=cv2.INTER_AREA)
        resized[i] = resized_roi.reshape(-1)

    # print(len(resized))

    return resized

# PyTorch式のデータセットクラスを定義

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, Y, transform=None):
        self.transform = transform
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        out_data = self.X[idx].reshape(1,64,64)
        out_data = torch.tensor(out_data, dtype=torch.float)

        #root_label = torch.tensor(self.Y[0][idx], dtype=torch.long)
        #vowel_label = torch.tensor(self.Y[1][idx], dtype=torch.long)
        #cons_label = torch.tensor(self.Y[2][idx], dtype=torch.long)

        root_label = torch.tensor(np.argmax(self.Y[0][idx]), dtype=torch.long)
        vowel_label = torch.tensor(np.argmax(self.Y[1][idx]), dtype=torch.long)
        cons_label = torch.tensor(np.argmax(self.Y[2][idx]), dtype=torch.long)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, root_label, vowel_label, cons_label

def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
     return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride,
                      padding=dilation, groups=groups, bias=True,
                      dilation=dilation)

def conv1x1(in_channels, out_channels, stride=1):
     return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True)

class BasicBlock(nn.Module):
    #Implementation of Basic Building Block

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()

        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity_x = x  # hold input for shortcut connection

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity_x = self.downsample(x)

        out += identity_x  # shortcut connection
        return self.relu(out)

class ResidualLayer(nn.Module):

    def __init__(self, num_blocks, in_channels, out_channels, block=BasicBlock):
        super(ResidualLayer, self).__init__()
        downsample = None
        if in_channels != out_channels:
            downsample = nn.Sequential(
                conv1x1(in_channels, out_channels),
                nn.BatchNorm2d(out_channels))
        self.first_block = block(in_channels, out_channels, downsample=downsample)
        self.blocks = nn.ModuleList(block(out_channels, out_channels) for _ in range(num_blocks))

    def forward(self, x):
        out = self.first_block(x)
        for block in self.blocks:
            out = block(out)
        return out

class model(nn.Module):
    def __init__(self):
        #resnet18の実装
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = ResidualLayer(2, in_channels=64, out_channels=64)
        self.layer2 = ResidualLayer(2, in_channels=64, out_channels=128)
        self.layer3 = ResidualLayer(2, in_channels=128, out_channels=256)
        self.layer4 = ResidualLayer(2, in_channels=256, out_channels=512)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 512)

        self.head_root = nn.Linear(512, 168) # + softmax
        self.head_vowel = nn.Linear(512, 11) # + softmax
        self.head_consonant = nn.Linear(512, 7) # + softmax
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        head_root = self.head_root(x)
        head_vowel = self.head_vowel(x)
        head_consonant = self.head_consonant(x)

        return head_root, head_vowel, head_consonant # not sure..

model = model()
model = try_gpu(model)

criterion1 = nn.CrossEntropyLoss() 
#criterion2 = nn.CrossEntropyLoss() 
#criterion3 = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

def train(model, epoch, train_loader):
    model.train()
    print(f'Epoch number {epoch}')
    correct_r, correct_v, correct_c = 0, 0, 0
    total_r, total_v, total_c = 0, 0, 0

    for i, data in enumerate(train_loader, 0):
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()

        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        total_r += root_l.size(0)
        correct_r += (root_pred == root_l).sum()
        total_v += vowel_l.size(0)
        correct_v += (vowel_pred == vowel_l).sum()
        total_c += consonant_l.size(0)
        correct_c += (consonant_pred == consonant_l).sum()
        # if i % 500 == 0:
        #     print("epoch{} root {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss1.data))
        #     print("epoch{} vowel {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss2.data))
        #     print("epoch{} consonant {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss3.data))
    
    print("root Accuracy {}/{} {:.2f}%".format(correct_r, total_r, 100.0*correct_r/total_r))
    print("vowel Accuracy {}/{} {:.2f}%".format(correct_v, total_v, 100.0*correct_v/total_v))
    print("consonant Accuracy {}/{} {:.2f}%".format(correct_c, total_c, 100.0*correct_c/total_c))


def test(model, test_loader):
    model.eval()
    correct_r, correct_v, correct_c = 0, 0, 0
    total_r, total_v, total_c = 0, 0, 0
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        total_r += root_l.size(0)
        correct_r += (root_pred == root_l).sum()
        total_v += vowel_l.size(0)
        correct_v += (vowel_pred == vowel_l).sum()
        total_c += consonant_l.size(0)
        correct_c += (consonant_pred == consonant_l).sum()

# 訓練ループ / Training Loop

val_perc = 0.2  # validation set の割合（クロスバリデーション）
epochs = 1 #4

#training loop
for parq_i in range(4):
    print('=============================')
    print(f'Parquet {parq_i} の訓練を開始')

    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)

    HEIGHT = 137
    WIDTH = 236

    X_train = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])

    X_train_resized = pd.DataFrame(resize(X_train)).T / 255.0   # 値を0~1におさめる
    X_train_resized = X_train_resized.values

    # メモリ節約
    del X_train

    # PyTorchのデータセットクラスを作る前に、ラベルの情報も整備
    # 1-of-K符号化とか、One Hot Encodingとか呼ばれる方法でラベルをつくる

    # 注：　PyTorchでは、ラベルはOne Hotじゃなくて良いことが判明したので、結局 MyDataset で元のラベルに戻している

    Y_train_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_train_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    Y_train = [Y_train_root, Y_train_vowel, Y_train_cons]

    trainval_dataset = MyDataset(X_train_resized, Y_train)

    n_samples = len(trainval_dataset)

    train_size = int(len(trainval_dataset)*(1.0 - val_perc))
    val_size = n_samples - train_size
    print(f'train size: {train_size}, validation size: {val_size}')

    train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

    train_loader = DataLoader(dataset=trainval_dataset,
                        batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=0)

    for i in range(1,epochs+1):
        train(model, i, train_loader)
        test(model, val_loader)

    # メモリ節約
    del train_df_with_img
    del train_dataset
    del trainval_dataset
    del train_loader
    del X_train_resized
    gc.collect()

# 提出ファイルの準備

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img)
    X_test_resized = pd.DataFrame(X_test_resized).T / 255.0   # 値を0~1におさめる
    X_test_resized = X_test_resized.values.reshape(-1, 1, 64, 64)

    test_inputs = torch.tensor(X_test_resized, dtype=torch.float)
    test_inputs = Variable(test_inputs)
    test_inputs = try_gpu(test_inputs)

    del X_test_resized
    gc.collect()
    
# テストデータをいっぺんに入れるとメモリが足りないので変更！
#     root_o, vowel_o, consonant_o = model(test_inputs)
#     root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
#     if torch.cuda.is_available():
#         root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))
#
#     root_pred, vowel_pred, consonant_pred = root_pred.numpy(), vowel_pred.numpy(), consonant_pred.numpy()
#
#     for k, id in enumerate(df_test_img.index.values):
#         row_id.append(id+'_consonant_diacritic')
#         target.append(consonant_pred[k])
#         row_id.append(id+'_grapheme_root')
#         target.append(root_pred[k])
#         row_id.append(id+'_vowel_diacritic')
#         target.append(vowel_pred[k])

    for k, id in enumerate(df_test_img.index.values):
        data = test_inputs[k].reshape(1,1,64,64)
        root_o, vowel_o, consonant_o = model(data)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        if torch.cuda.is_available():
            root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))

        root_pred, vowel_pred, consonant_pred = root_pred.item(), vowel_pred.item(), consonant_pred.item()
        
        row_id.append(id+'_consonant_diacritic')
        target.append(consonant_pred)
        row_id.append(id+'_grapheme_root')
        target.append(root_pred)
        row_id.append(id+'_vowel_diacritic')
        target.append(vowel_pred)
    
    del df_test_img
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

df_sample.to_csv('submission.csv',index=False)
df_sample.head(36)

Parquet 0 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 5373/50210 10.70%
vowel Accuracy 33992/50210 67.70%
consonant Accuracy 37090/50210 73.87%
Parquet 1 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 26954/50210 53.68%
vowel Accuracy 44123/50210 87.88%
consonant Accuracy 45554/50210 90.73%
Parquet 2 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 37933/50210 75.55%
vowel Accuracy 46639/50210 92.89%
consonant Accuracy 47014/50210 93.63%
Parquet 3 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 41119/50210 81.89%
vowel Accuracy 47413/50210 94.43%
consonant Accuracy 47664/50210 94.93%
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…


,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,1
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0
